In [ ]:
# First, try downloading from drive directly.
!pip install -U --no-cache-dir gdown --pre
!gdown --fuzzy "google-drive-share-link"

In [8]:
# If above does not work, use the access token approach
# Instruction on how to get the token: https://stackoverflow.com/a/67550427
access_token = "" # your access token
file_id = "" # your file id
output_name = "nabirds_yolo_cleaned.zip"

!curl -H "Authorization: Bearer {access_token}" \
     "https://www.googleapis.com/drive/v3/files/{file_id}?alt=media" \
     -o {output_name}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9512M  100 9512M    0     0  34.0M      0  0:04:39  0:04:39 --:--:-- 33.8MM   98 9373M    0     0  34.0M      0  0:04:39  0:04:35  0:00:04 30.9M


In [ ]:
import os
import shutil

# --- CONFIGURATION ---
WORK_DIR = "/workspace"
ZIP_FILENAME = "nabirds_yolo_cleaned.zip"
DATASET_FOLDER_NAME = "nabirds_yolo_cleaned"

# Paths
ZIP_PATH = os.path.join(WORK_DIR, ZIP_FILENAME)
DATASET_DIR = os.path.join(WORK_DIR, DATASET_FOLDER_NAME)

# Where to save training outputs
PROJECT_ROOT = os.path.join(WORK_DIR, "YOLO_Training_Runs")
PROJECT_NAME = "nabirds_v1"
# ---------------------

# 1. Install 'unzip' utility (Fix for "command not found")
print("🛠️ Installing system tools...")
!apt-get update -qq && apt-get install -y unzip -qq

# 2. Unzip Logic
if not os.path.exists(DATASET_DIR):
    if os.path.exists(ZIP_PATH):
        print(f"📦 Found {ZIP_FILENAME}. Unzipping...")
        !unzip -q {ZIP_PATH} -d {WORK_DIR}
        print("✅ Data ready.")
    else:
        print(f"❌ Error: {ZIP_FILENAME} not found in {WORK_DIR}. Please upload it first!")
else:
    print("✅ Dataset folder already exists. Skipping unzip.")

🛠️ Installing system tools...
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package unzip.
(Reading database ... 24135 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-26ubuntu3.2_amd64.deb ...
Unpacking unzip (6.0-26ubuntu3.2) ...
Setting up unzip (6.0-26ubuntu3.2) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
📦 Found nabirds_yolo_cleaned.zip. Unzipping...


In [ ]:
# RunPod templates usually have torch, but we need ultralytics
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
# Path to the resume checkpoint
ckpt_path = os.path.join(PROJECT_ROOT, PROJECT_NAME, "weights", "last.pt")

if os.path.exists(ckpt_path):
    print(f"🔄 Found checkpoint at {ckpt_path}.")
    print("Resuming training... (Appending to results.csv)")
    
    # Load previous state
    model = YOLO(ckpt_path)
    
    # RESUME: strictly no other arguments allowed
    model.train(resume=True)

else:
    print("🆕 No previous checkpoint found. Starting FRESH training...")
    
    # Load base model
    model = YOLO("yolo11n.pt")
    
    # Start Training
    model.train(
        data=os.path.join(DATASET_DIR, "dataset.yaml"),
        epochs=100,
        imgsz=640,
        batch=64,              # Auto-batch
        patience=15,
        project=PROJECT_ROOT,  # Save to /workspace/YOLO_Training_Runs
        name=PROJECT_NAME,
        exist_ok=True,
        device=0,              # First GPU
        workers=6,             # RunPod CPUs are beefy, use more workers
        compile=True,
        cos_lr=True
    )

In [ ]:
# Zip the training results folder
output_zip = os.path.join(WORK_DIR, f"{PROJECT_NAME}_results.zip")
source_dir = os.path.join(PROJECT_ROOT, PROJECT_NAME)

print(f"Zipping results from {source_dir}...")
shutil.make_archive(output_zip.replace('.zip', ''), 'zip', source_dir)

print(f"✅ Ready! Download '{PROJECT_NAME}_results.zip' from the file browser on the left.")